In [1]:
!pip install autogluon

In [88]:
import os
import pandas as pd
import random
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) 

In [89]:
train = pd.read_csv("C:/Users/user/Desktop/흡연여부/open/train.csv")
test = pd.read_csv("C:/Users/user/Desktop/흡연여부/open/test.csv")
sample_submission= pd.read_csv("C:/Users/user/Desktop/흡연여부/open/sample_submission.csv")


In [90]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7000 non-null   object 
 1   나이        7000 non-null   int64  
 2   키(cm)     7000 non-null   int64  
 3   몸무게(kg)   7000 non-null   int64  
 4   BMI       7000 non-null   float64
 5   시력        7000 non-null   float64
 6   충치        7000 non-null   int64  
 7   공복 혈당     7000 non-null   int64  
 8   혈압        7000 non-null   int64  
 9   중성 지방     7000 non-null   int64  
 10  혈청 크레아티닌  7000 non-null   float64
 11  콜레스테롤     7000 non-null   int64  
 12  고밀도지단백    7000 non-null   int64  
 13  저밀도지단백    7000 non-null   int64  
 14  헤모글로빈     7000 non-null   float64
 15  요 단백      7000 non-null   int64  
 16  간 효소율     7000 non-null   float64
 17  label     7000 non-null   int64  
dtypes: float64(5), int64(12), object(1)
memory usage: 984.5+ KB


In [91]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        3000 non-null   object 
 1   나이        3000 non-null   int64  
 2   키(cm)     3000 non-null   int64  
 3   몸무게(kg)   3000 non-null   int64  
 4   BMI       3000 non-null   float64
 5   시력        3000 non-null   float64
 6   충치        3000 non-null   int64  
 7   공복 혈당     3000 non-null   int64  
 8   혈압        3000 non-null   int64  
 9   중성 지방     3000 non-null   int64  
 10  혈청 크레아티닌  3000 non-null   float64
 11  콜레스테롤     3000 non-null   int64  
 12  고밀도지단백    3000 non-null   int64  
 13  저밀도지단백    3000 non-null   int64  
 14  헤모글로빈     3000 non-null   float64
 15  요 단백      3000 non-null   int64  
 16  간 효소율     3000 non-null   float64
dtypes: float64(5), int64(11), object(1)
memory usage: 398.6+ KB


In [92]:
train = train.drop(['ID'], axis = 1)
test = test.drop('ID', axis = 1)

In [93]:
train['혈청 크레아티닌'].describe()

count    7000.000000
mean        0.884900
std         0.241523
min         0.100000
25%         0.800000
50%         0.900000
75%         1.000000
max        10.000000
Name: 혈청 크레아티닌, dtype: float64

In [94]:
# 파생변수 =>혈청 크레아티닌값에 따라 정상/비정상 여부를 판단하여 1 또는 0으로 기록
train['혈청 크레아티닌_status'] = train['혈청 크레아티닌'].apply(lambda x: 1 if 0.8 <= x <= 1.7 else 0)


In [95]:
## 상관분석

import scipy.stats as stats

PSI_corr = pd.DataFrame()
PSI_corr

Y = train['label'].values

for item in train.drop(columns=['label']).columns :
    X = train[item].values
    new_data = {
        'variable' : [item],
        'Correlation' : [round(stats.pearsonr(X, Y)[0], 2)],
        'p-value' : [round(stats.pearsonr(X, Y)[1], 3)]
    }
   
    new_df = pd.DataFrame(new_data)
    PSI_corr = pd.concat([PSI_corr, new_df])

PSI_corr = PSI_corr.reset_index(drop=True)
PSI_corr

,variable,Correlation,p-value
0,나이,-0.17,0.000
1,키(cm),0.39,0.000
2,몸무게(kg),0.31,0.000
3,BMI,0.13,0.000
4,시력,0.07,0.000
5,충치,0.10,0.000
6,공복 혈당,0.11,0.000
7,혈압,0.02,0.110
8,중성 지방,0.25,0.000
9,혈청 크레아티닌,0.20,0.000


In [96]:
j = len(PSI_corr)
PSI_corr = PSI_corr.reset_index(drop=True)

for i in range(0, j) :
    if PSI_corr['p-value'][i] >= 0.05 :
        PSI_corr = PSI_corr.drop(i)
#     else :
#         if PSI_corr['Correlation'][i] > -0.35 and PSI_corr['Correlation'][i] < 0.35 :
#             PSI_corr = PSI_corr.drop(i)

PSI_corr = PSI_corr.sort_values('Correlation', ascending = False)
PSI_corr = PSI_corr.reset_index(drop=True)
PSI_corr

,variable,Correlation,p-value
0,헤모글로빈,0.40,0.000
1,키(cm),0.39,0.000
2,몸무게(kg),0.31,0.000
3,중성 지방,0.25,0.000
4,혈청 크레아티닌_status,0.25,0.000
5,혈청 크레아티닌,0.20,0.000
6,BMI,0.13,0.000
7,공복 혈당,0.11,0.000
8,충치,0.10,0.000
9,시력,0.07,0.000


In [97]:
#x_train, y_train 설정- 상관분석 고려한거
train = train.drop(['콜레스테롤', '요 단백'], axis=1)


In [98]:
# 학습데이터에 했던 전처리 과정 테스트데이터에도 진행
test['혈청 크레아티닌_status'] = test['혈청 크레아티닌'].apply(lambda x: 1 if 0.8 <= x <= 1.7 else 0)
test = test.drop(['콜레스테롤', '요 단백'], axis=1)

In [120]:
from sklearn.model_selection import train_test_split, KFold

test_data = TabularDataset(test)
kfold = KFold(n_splits=10, shuffle=True)
pres = np.zeros(len(test))
for fold, (train_idx, val_idx) in enumerate(kfold.split(train)):
    print(f'=========={fold+1}fold/10 folds==========')
    train_data = train.copy()
    train_x = train_data.iloc[train_idx]
    val_x = train_data.iloc[val_idx]
    
    train_x =TabularDataset(train_x)
    val_x =TabularDataset(val_x)
    
    predictor = TabularPredictor(
        label=label, eval_metric=eval_metric
    ).fit(train_x,val_x)  
    pre = predictor.predict(test_data)
    pres += (pre/10)
    
sample_submission['label'] = pres.round().astype(int)
 
sample_submission.to_csv("C:/Users/user/Desktop/흡연여부/open/sub_10fold.csv",index=False)
sample_submission

No path specified. Models will be saved in: "AutogluonModels\ag-20230826_174336\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230826_174336\"
AutoGluon Version:  0.8.2
Python Version:     3.8.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   65.75 GB / 237.11 GB (27.7%)
Train Data Rows:    6300
Train Data Columns: 15
Tuning Data Rows:    700
Tuning Data Columns: 15
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	

==========1fold/10 folds==========


	0.67	 = Validation score   (accuracy)
	0.02s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.68	 = Validation score   (accuracy)
	0.03s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.73	 = Validation score   (accuracy)
	0.43s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ...
	0.72	 = Validation score   (accuracy)
	0.41s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7086	 = Validation score   (accuracy)
	1.18s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.71	 = Validation score   (accuracy)
	1.27s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ...
	0.7114	 = Validation score   (accuracy)
	2.28s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7143	 = Validation score   (accuracy)
	1.0s	 = Training   runtime
	0.1s	 = Validation runt

==========2fold/10 folds==========


	0.6586	 = Validation score   (accuracy)
	0.03s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.67	 = Validation score   (accuracy)
	0.04s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.73	 = Validation score   (accuracy)
	0.49s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ...
	0.7229	 = Validation score   (accuracy)
	0.93s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7186	 = Validation score   (accuracy)
	1.23s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7143	 = Validation score   (accuracy)
	1.43s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	0.7286	 = Validation score   (accuracy)
	2.12s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7014	 = Validation score   (accuracy)
	1.05s	 = Training   runtime
	0.11s	 = Validati

...

==========9fold/10 folds==========


	0.6714	 = Validation score   (accuracy)
	0.03s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.6843	 = Validation score   (accuracy)
	0.04s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.7514	 = Validation score   (accuracy)
	0.81s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.7557	 = Validation score   (accuracy)
	0.58s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7486	 = Validation score   (accuracy)
	1.28s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7471	 = Validation score   (accuracy)
	1.44s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	0.7457	 = Validation score   (accuracy)
	2.81s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7414	 = Validation score   (accuracy)
	1.05s	 = Training   runtime
	0.13s	 = Val

==========10fold/10 folds==========


	0.6514	 = Validation score   (accuracy)
	0.03s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.6671	 = Validation score   (accuracy)
	0.02s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.7429	 = Validation score   (accuracy)
	0.83s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.7486	 = Validation score   (accuracy)
	0.53s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7429	 = Validation score   (accuracy)
	1.32s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7371	 = Validation score   (accuracy)
	1.51s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: CatBoost ...
	0.7357	 = Validation score   (accuracy)
	5.29s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7214	 = Validation score   (accuracy)
	1.09s	 = Training   runtime
	0.12s	 = Va

,ID,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,1
4,TEST_0004,0
...,...,...
2995,TEST_2995,0
2996,TEST_2996,0
2997,TEST_2997,0
2998,TEST_2998,1


In [119]:
from sklearn.model_selection import train_test_split, KFold

test_data = TabularDataset(test)
kfold = KFold(n_splits=30, shuffle=True)
pres = np.zeros(len(test))
for fold, (train_idx, val_idx) in enumerate(kfold.split(train)):
    print(f'=========={fold+1}fold/30 folds==========')
    train_data = train.copy()
    train_x = train_data.iloc[train_idx]
    val_x = train_data.iloc[val_idx]
    
    train_x =TabularDataset(train_x)
    val_x =TabularDataset(val_x)
    
    predictor = TabularPredictor(
        label=label, eval_metric=eval_metric
    ).fit(train_x,val_x)  
    pre = predictor.predict(test_data)
    pres += (pre/30)
    
sample_submission['label'] = pres.round().astype(int)
 
sample_submission.to_csv("C:/Users/user/Desktop/흡연여부/open/sub_30fold.csv",index=False)
sample_submission


No path specified. Models will be saved in: "AutogluonModels\ag-20230826_171814\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230826_171814\"
AutoGluon Version:  0.8.2
Python Version:     3.8.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   74.00 GB / 237.11 GB (31.2%)
Train Data Rows:    6766
Train Data Columns: 15
Tuning Data Rows:    234
Tuning Data Columns: 15
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	

==========1fold/30 folds==========


	0.6624	 = Validation score   (accuracy)
	0.02s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.6624	 = Validation score   (accuracy)
	0.03s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.7009	 = Validation score   (accuracy)
	0.47s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ...
	0.6838	 = Validation score   (accuracy)
	0.44s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.6752	 = Validation score   (accuracy)
	1.23s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7009	 = Validation score   (accuracy)
	1.52s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost ...
	0.6752	 = Validation score   (accuracy)
	2.92s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.6538	 = Validation score   (accuracy)
	0.97s	 = Training   runtime
	0.07s	 = Vali

==========2fold/30 folds==========


	0.03s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.6795	 = Validation score   (accuracy)
	0.03s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.7308	 = Validation score   (accuracy)
	0.46s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ...
	0.7265	 = Validation score   (accuracy)
	0.45s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7179	 = Validation score   (accuracy)
	1.29s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7094	 = Validation score   (accuracy)
	1.4s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: CatBoost ...
	0.7308	 = Validation score   (accuracy)
	1.98s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.6923	 = Validation score   (accuracy)
	1.08s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEn

...

==========29fold/30 folds==========


	0.02s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.7082	 = Validation score   (accuracy)
	0.03s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.7554	 = Validation score   (accuracy)
	0.99s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.7854	 = Validation score   (accuracy)
	0.9s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7597	 = Validation score   (accuracy)
	1.36s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7811	 = Validation score   (accuracy)
	1.52s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	0.7854	 = Validation score   (accuracy)
	6.66s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7382	 = Validation score   (accuracy)
	1.08s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTrees

==========30fold/30 folds==========


	0.6395	 = Validation score   (accuracy)
	0.02s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.6481	 = Validation score   (accuracy)
	0.03s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.7382	 = Validation score   (accuracy)
	0.68s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ...
	0.7425	 = Validation score   (accuracy)
	0.45s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7511	 = Validation score   (accuracy)
	1.36s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.721	 = Validation score   (accuracy)
	1.55s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost ...
	0.7639	 = Validation score   (accuracy)
	2.05s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7253	 = Validation score   (accuracy)
	1.04s	 = Training   runtime
	0.11s	 = Valid

,ID,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,1
4,TEST_0004,0
...,...,...
2995,TEST_2995,0
2996,TEST_2996,0
2997,TEST_2997,0
2998,TEST_2998,1


In [133]:
train_data = TabularDataset(train)
test_data = TabularDataset(test)

label = 'label'
eval_metric = 'accuracy'
time_limit = 1200

In [134]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data,  presets='best_quality', time_limit=time_limit, 
      num_stack_levels=5, num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels\ag-20230827_014620\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=5, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1200s
AutoGluon will save models to "AutogluonModels\ag-20230827_014620\"
AutoGluon Version:  0.8.2
Python Version:     3.8.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   58.38 GB / 237.11 GB (24.6%)
Train Data Rows:    7000
Train Data Columns: 15
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 =

Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.757	 = Validation score   (accuracy)
	4.69s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 257.27s of the 1005.73s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.7583	 = Validation score   (accuracy)
	6.36s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L2 ... Training model for up to 240.08s of the 988.54s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	0.7489	 = Validation score   (accuracy)
	2.87s	 = Training   runtime
	0.51s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L2 ... Training model for up to 236.56s of the 985.03s of rema

Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.7756	 = Validation score   (accuracy)
	4.8s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L4 ... Training model for up to 249.07s of the 600.95s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	0.7703	 = Validation score   (accuracy)
	3.42s	 = Training   runtime
	0.43s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L4 ... Training model for up to 245.08s of the 596.96s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	0.7703	 = Validation score   (accuracy)
	3.75s	 = Training   runtime
	0.42s	 = Validation runtime
Fitting model: CatBoost_BAG_L4 ... Training model for up to 240.78s of the 592.66s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead


Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	0.7697	 = Validation score   (accuracy)
	2.96s	 = Training   runtime
	0.54s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L6 ... Training model for up to 214.49s of the 214.47s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	0.7741	 = Validation score   (accuracy)
	3.72s	 = Training   runtime
	0.42s	 = Validation runtime
Fitting model: CatBoost_BAG_L6 ... Training model for up to 210.21s of the 210.2s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.7803	 = Validation score   (accuracy)
	11.91s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini_BAG_L6 ... Training model for up to 187.78s of the 187.76s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	

In [135]:
predictor.leaderboard(silent = True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L6,0.780286,14.637513,558.705616,0.030702,11.907441,6,True,67
1,WeightedEnsemble_L7,0.780286,14.660514,562.483496,0.023000,3.777881,7,True,74
2,LightGBMXT_BAG_L6,0.779571,14.710610,553.266048,0.103799,6.467874,6,True,63
3,WeightedEnsemble_L6,0.778857,12.032406,463.648183,0.017911,3.610619,6,True,62
4,LightGBM_BAG_L6,0.778857,14.646325,553.596279,0.039513,6.798105,6,True,64
...,...,...,...,...,...,...,...,...,...
69,RandomForestGini_BAG_L1,0.736000,0.394171,2.044832,0.394171,2.044832,1,True,5
70,ExtraTreesGini_BAG_L1,0.725571,0.409451,1.200777,0.409451,1.200777,1,True,8
71,ExtraTreesEntr_BAG_L1,0.725143,0.431567,1.138430,0.431567,1.138430,1,True,9
72,KNeighborsDist_BAG_L1,0.684000,0.140397,0.022281,0.140397,0.022281,1,True,2


In [136]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)

In [137]:
sample_submission['label'] = model_pred
sample_submission

,ID,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,1
4,TEST_0004,0
...,...,...
2995,TEST_2995,0
2996,TEST_2996,0
2997,TEST_2997,0
2998,TEST_2998,0


In [138]:
sample_submission.to_csv("C:/Users/user/Desktop/흡연여부/open/submission20minute.csv",index=False)

In [147]:
train_data = TabularDataset(train)
test_data = TabularDataset(test)

label = 'label'
eval_metric = 'accuracy'
time_limit = 1200

In [148]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data,  presets='best_quality', time_limit=time_limit, 
      num_stack_levels=3, num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels\ag-20230827_024954\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1200s
AutoGluon will save models to "AutogluonModels\ag-20230827_024954\"
AutoGluon Version:  0.8.2
Python Version:     3.8.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   55.45 GB / 237.11 GB (23.4%)
Train Data Rows:    7000
Train Data Columns: 15
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 =

Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.757	 = Validation score   (accuracy)
	6.81s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 437.19s of the 1005.71s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.7583	 = Validation score   (accuracy)
	7.18s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L2 ... Training model for up to 420.0s of the 988.51s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	0.7489	 = Validation score   (accuracy)
	2.83s	 = Training   runtime
	0.47s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L2 ... Training model for up to 416.53s of the 985.05s of rema

	0.7524	 = Validation score   (accuracy)
	1.17s	 = Training   runtime
	0.48s	 = Validation runtime
Fitting model: ExtraTreesEntr_BAG_L3 ... Training model for up to 367.66s of the 584.96s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	0.7537	 = Validation score   (accuracy)
	1.15s	 = Training   runtime
	0.42s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L3 ... Training model for up to 365.93s of the 583.23s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.7551	 = Validation score   (accuracy)
	23.22s	 = Training   runtime
	0.32s	 = Validation runtime
Fitting model: XGBoost_BAG_L3 ... Training model for up to 332.29s of the 549.58s of remaining time.
Specified total num_gpus: 1, but only 0 are available. Will use 0 instead
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittin

	0.7597	 = Validation score   (accuracy)
	13.84s	 = Training   runtime
	0.19s	 = Validation runtime
Completed 1/20 k-fold bagging repeats ...
Fitting model: WeightedEnsemble_L5 ... Training model for up to 360.0s of the 105.82s of remaining time.
	0.7666	 = Validation score   (accuracy)
	3.47s	 = Training   runtime
	0.01s	 = Validation runtime
AutoGluon training complete, total runtime = 1097.71s ... Best model: "WeightedEnsemble_L5"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20230827_024954\")


In [149]:
predictor.leaderboard(silent = True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_BAG_L4,0.766571,11.013432,516.730823,0.084721,6.098360,4,True,47
1,WeightedEnsemble_L5,0.766571,11.027386,520.200598,0.013954,3.469775,5,True,50
2,CatBoost_BAG_L4,0.764000,10.949810,525.019821,0.021100,14.387357,4,True,43
3,LightGBMXT_BAG_L4,0.763714,11.049625,517.407499,0.120914,6.775036,4,True,39
4,CatBoost_BAG_L3,0.762857,7.337751,339.788736,0.045501,27.237652,3,True,31
5,WeightedEnsemble_L4,0.762857,7.352705,343.428763,0.014953,3.640027,4,True,38
6,WeightedEnsemble_L3,0.762286,4.807809,209.263709,0.024670,4.048929,3,True,26
7,LightGBM_BAG_L4,0.762143,10.994237,517.666622,0.065526,7.034158,4,True,40
8,NeuralNetTorch_BAG_L4,0.760714,11.136597,541.010987,0.207886,30.378523,4,True,48
9,LightGBMLarge_BAG_L4,0.759714,11.121927,524.475378,0.193216,13.842915,4,True,49


In [150]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)

In [151]:
sample_submission['label'] = model_pred
sample_submission

,ID,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,1
4,TEST_0004,0
...,...,...
2995,TEST_2995,0
2996,TEST_2996,0
2997,TEST_2997,0
2998,TEST_2998,0


In [152]:
sample_submission.to_csv("C:/Users/user/Desktop/흡연여부/open/submission20minute3stack.csv",index=False)